In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import nltk
nltk.download("stopwords")
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm.notebook import tqdm,tnrange
import glob
import os
import json
import string
from functools import reduce
from tabulate import tabulate
from collections import defaultdict
dictionary={}#dictionary 


Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocessing(file):
  file=file.lower()
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  tokenized_text=tokenizer.tokenize(file)
  stop_words = stopwords.words('english')
  filtered_text = [i for i in tokenized_text if i not in stop_words]
  validTokens = [''.join(ch for ch in x if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation) for x in filtered_text]
  preprocessedFile = [x for x in validTokens if len(x) > 1]
  return preprocessedFile

In [ ]:
def indexFile(text,docId):
  for pos,val in enumerate(text):
    if val in dictionary:
       dictionary[val][0]+=1
       if(docId in dictionary[val][1]):
         dictionary[val][1][docId].append(pos)
       else:
          dictionary[val][1][docId]=[pos]
    else:
      dictionary[val]=[1,{docId:[pos]}]


In [ ]:
files = os.walk("/content/drive/MyDrive/IR/Assignment-1/Humor,Hist,Media,Food")
filePaths = []
for i in files:
    for j in i[2]:
        filePath = i[0] + "/" + j
        filePaths.append(filePath)

json.dump(filePaths, open("path_mapping.json", "w"))

In [ ]:
for i,filePath in enumerate(tqdm(filePaths)):
    try:
        file = open(filePath, encoding="utf8")
        read = file.read().replace('\n', ' ')    
    except Exception as e:
        file = open(filePath, encoding="unicode_escape")
        read = file.read().replace('\n', ' ')
    file.close()
    read=preprocessing(read)
    indexFile(read, i)
    json.dump(dictionary, open('dictionary.json',"w"))
    

  0%|          | 0/1133 [00:00<?, ?it/s]

# Query

In [ ]:
def commonDocIds(docIds):       
  if len(docIds)==0:
      return []
  
  docIds.sort(key=len)
  return list(reduce(lambda x,y: set(x)&set(y),docIds))

In [ ]:
def getMapping(files):
  print('Total Documents:',len(files))
  mapping=json.load(open('/content/drive/MyDrive/IR/Assignment-1/path_mapping.json'))
  data= list(map(lambda i:(i,mapping[i]),files)) # list of tuples consisting of document id and their location
  data.sort()  
  print(tabulate(data,headers=['Document ID','Location']))

In [ ]:
def phraseQuery(phrase):
  phrase=preprocessing(phrase)
  for word in phrase:
    #if term is not present in dictionary then return empty list 
    if word not in dictionary:
      return []
  
  if(len(phrase)==1):
    result=list(dictionary[phrase[0]][1].keys())
    print(result)
  else:
      postingList=[]
      for words in phrase:
        temp=[]
        for i in dictionary[words][1]:
          temp.append([i,dictionary[words][1][i]])
        postingList.append(temp)
      docIds=[ [x[0] for x in p] for p in postingList ]
      docIds= commonDocIds(docIds)
      
      for i in range(len(postingList)):
        temp=[]
        for id in postingList[i]:
          if id[0] in docIds:
            temp.append(id)
        postingList[i]=temp    

      for i in range(len(postingList)):
        for j in range(len(postingList[i])):
          temp=[]
          for k in postingList[i][j][1]:
            temp.append(k-i)
          postingList[i][j][1]=temp

      result=[]
      for i in range(len(postingList[0])):
        intersection=commonDocIds([x[i][1] for x in postingList])
        if len(intersection)>0:
          result.append(postingList[0][i][0]) 

  result=list(map(int,result))
  print('Total Documents:',len(result))
  mapping=json.load(open('/content/drive/MyDrive/IR/Assignment-1/path_mapping.json'))
  data= list(map(lambda i:(i,mapping[i]),result)) # list of tuples consisting of document id and their location
  data.sort()  
  print(tabulate(data,headers=['Document ID','Location']))

In [ ]:
dictionary=json.load(open("/content/drive/MyDrive/IR/Assignment-1/dictionary.json"))
dictionary=defaultdict(lambda:[],dictionary)
phrase=input()
output=phraseQuery(phrase)

patient requiring  intensive  care
Total Documents: 1
  Document ID  Location
-------------  --------------------------------------------------------------------------------
          439  /content/drive/MyDrive/IR/Humor,Hist,Media,Food/Humor,Hist,Media,Food/aclamt.txt
